# Preperation

## ClearML

In [1]:
# https://app.clear.ml/projects/4f45f06bb51f43ecb10dcaf8319c3e76/experiments/a9b2315cdf234266808858d91d90cc74/output/log

from clearml import Task
task = Task.init(project_name='count-pepper', task_name='train')

ClearML Task: created new task id=2ea85ecc80744502823b2cd800e657c4
2024-11-05 17:17:35,610 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-11-05 17:17:35,612 - clearml.Repository Detection - WARNING - Please install nbconvert using "pip install nbconvert"
2024-11-05 17:17:35,616 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/1aa0d6c5dab5447095488a6887d3fff8/experiments/2ea85ecc80744502823b2cd800e657c4/output/log


## Roboflow

In [2]:
import os, yaml
from credentials import credentials
from roboflow import Roboflow

# https://universe.roboflow.com/iit-kgp-knvbv/field-capsicum/dataset/1

rf = Roboflow(api_key=credentials.get('ROBOFLOW-API-KEY'))
project = rf.workspace("andreis-workspace").project("green-pepper-detection")
version = project.version(1)
dataset = version.download("yolov8")

directory = dataset.location
new_directory_name = directory.rsplit('/', 1)[0] + '/dataset'

os.rename(directory, new_directory_name)

config_file_path = './dataset/data.yaml'

with open(config_file_path, 'r') as f:
    config_file = yaml.safe_load(f)

config_file['path'] = new_directory_name
config_file['test'] = "test/images"
config_file['train'] = "train/images"
config_file['val'] = "valid/images"

with open(config_file_path, "w") as f:
    yaml.dump(config_file, f)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.74, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Green-Pepper-Detection-1 in yolov8:: 100%|██████████| 1448/1448 [00:00<00:00, 6643.11it/s]


# Machine Learning

## Load Model

In [3]:
from ultralytics import YOLO

try:
    model = YOLO('./pepper-detection.pt') # load a custom model
except FileNotFoundError:
    model = YOLO('yolov8n.pt') # load a pretrained model


2024-11-05 17:17:56,764 - clearml.model - INFO - Selected model id: 7ef67599c613491bb658498a23bd8eb7


## Train

### From scratch

In [4]:
from ultralytics import YOLO

# Train the model
results = model.train(data="./dataset/data.yaml", epochs=95, batch=-1, device=[0])

New https://pypi.org/project/ultralytics/8.3.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.74 🚀 Python-3.10.14 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./dataset/data.yaml, epochs=95, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False

/home/takada-labo/projects/count-pepper/.venv/lib/python3.10/site-packages/ultralytics/engine/trainer.py:269: FutureWarning:

`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.



     3011043       8.194         0.201         26.37         81.12        (1, 3, 640, 640)                    list
     3011043       16.39         0.298         20.88         24.87        (2, 3, 640, 640)                    list
     3011043       32.78         0.526         21.38         25.45        (4, 3, 640, 640)                    list
     3011043       65.55         1.038         22.45          29.5        (8, 3, 640, 640)                    list
     3011043       131.1         1.994         31.52         35.64       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 38 for CUDA:0 4.82G/8.00G (60%) ✅


train: Scanning /home/takada-labo/projects/count-pepper/dataset/train/labels... 624 images, 41 backgrounds, 0 corrupt: 100%|██████████| 624/624 [00:00<00:00, 965.84it/s]

train: New cache created: /home/takada-labo/projects/count-pepper/dataset/train/labels.cache


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.12it/s]


                   all         63        814      0.825      0.555      0.647      0.417
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train


2024-11-05 17:27:15,853 - clearml.storage - INFO - Uploading: 5.96MB to runs/detect/train/weights/best.pt


████████████████████████████████▉ 100% | 5.96/5.96 MB [00:04<00:00,  1.38MB/s]: 

2024-11-05 17:27:20,192 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/count-pepper/train.2ea85ecc80744502823b2cd800e657c4/models/best.pt


### Resume training

In [4]:
import glob
from ultralytics import YOLO

train_folders = glob.glob('./runs/detect/train*')
latest_number = 0
for train_folder in train_folders:
    number = int(0 if train_folder.split('train')[1] == '' else train_folder.split('train')[1])
    latest_number = max(latest_number, number)

last_model_path = f'./runs/detect/train{latest_number}/weights/last.pt' if latest_number != 0 else f'./runs/detect/train/weights/last.pt'

model = YOLO(last_model_path)

# results = model.train(resume=True, batch=-1, device=[0])

## Validate

In [ ]:
from ultralytics import YOLO

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
print(metrics.box.map)  # map50-95

Ultralytics YOLOv8.2.52 🚀 Python-3.10.14 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/arcdev/follower-tello-drone/Head_Detection-1/valid/labels.cache... 3529 images, 36 backgrounds, 0 corrupt: 100%|██████████| 3529/3529 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/arcdev/follower-tello-drone/Head_Detection-1/valid/images/1964_jpg.rf.343b2da041c2c2f58a359ddc214c5f15.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /home/arcdev/follower-tello-drone/Head_Detection-1/valid/images/Kamera_17_part_1689243468000_1689244068000_mp4-35_jpg.rf.04389a18dd781138c648df5f181f81cc.jpg: 10 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 221/221 [00:36<00:00,  6.02it/s]


                   all       3529      29429      0.868      0.731      0.807      0.393
Speed: 0.3ms preprocess, 4.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/arcdev/follower-tello-drone/runs/detect/val
0.3933452256556003


## Export

In [5]:
import glob
from ultralytics import YOLO

train_folders = glob.glob('./runs/detect/train*')
latest_number = 0
for train_folder in train_folders:
    number = int(0 if train_folder.split('train')[1] == '' else train_folder.split('train')[1])
    latest_number = max(latest_number, number)

best_model_path = f'./runs/detect/train{latest_number}/weights/last.pt' if latest_number != 0 else f'./runs/detect/train/weights/last.pt'

model = YOLO(best_model_path)

model.save("pepper-detection.pt")